In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import shfl
import torch
import copy
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0-cropped", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 100,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
lambda_values = [float(10**(-n)) for n in range(1, 10)]

print(lambda_values)

[0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07, 1e-08, 1e-09]


In [3]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder():
    return ClassifierModel(batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels

    t_test_data, t_test_label = cit_federated_government.global_model.transform_data(test_data, test_label, lb1, lb2)

    return t_federated_data, t_test_data, t_test_label

In [4]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

data, label, train_data, train_label, test_data, test_label, train_files, test_files = get_data_csv(args["data_path"], args["csv_path"], lb1)

print(len(train_data))
print(len(test_data))

16
4


In [5]:
cit_model = cit_builder()
cit_model.train(train_data, train_label)

t_train_data, t_train_label = cit_model.transform_data(train_data, train_label, lb1, lb2)
t_test_data, t_test_label = cit_model.transform_data(test_data, test_label, lb1, lb2)

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] weights = [1. 1.]
LAMBDA: 0.05
FOLD: 0


  0%|          | 0/10 [00:00<?, ?it/s]

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING


[1/2] Loss_D: 0.5534 Acc_D: 0.4500 Loss_G_class1: 0.3941 Loss_G_class2: 0.2606: 100%|██████████| 10/10 [00:02<00:00,  4.15it/s]
[Validating]: Acc_D: 0.5000: 100%|██████████| 6/6 [00:00<00:00, 14.05it/s]


Valid Loss = 1.0360544870297115
Valid Acc = 0.5


...

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING


[1/2] Loss_D: 0.5562 Acc_D: 0.4545 Loss_G_class1: 0.5824 Loss_G_class2: 0.4605: 100%|██████████| 11/11 [00:02<00:00,  4.34it/s]
  0%|          | 0/11 [00:00<?, ?it/s]

Valid Loss = 1.7610604286193847
Valid Acc = 0.4


[2/2] Loss_D: 0.3827 Acc_D: 0.4091 Loss_G_class1: 0.5719 Loss_G_class2: 0.4607: 100%|██████████| 11/11 [00:02<00:00,  4.32it/s]
[Validating]: Acc_D: 0.6000: 100%|██████████| 5/5 [00:00<00:00, 12.80it/s]


Valid Loss = 0.7169366955757142
Valid Acc = 0.6
BEST LAMBDA: 0.05


In [6]:
from ClassifierModel import ClassifierModel

classifier_model = classifier_builder()
classifier_model.train(t_train_data, t_train_label)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/2
25/25 [==============================] - 17s 678ms/step - loss: 1.6695 - categorical_accuracy: 0.3200 - val_loss: 1.5236 - val_categorical_accuracy: 0.3333
Epoch 2/2
25/25 [==============================] - 1s 54ms/step - loss: 1.5135 - categorical_accuracy: 0.2800 - val_loss: 1.6180 - val_categorical_accuracy: 0.3333


In [7]:
dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 
G_dict = cit_model._G_dict

for key, _ in G_dict.items():
    G_dict[key].to("cpu")

classifier_model.get_classification_report(test_files, dict_labels, G_dict)

Accuracy 4 clases: 0.25
Numero de veces no concuerda: 4
{'N': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 2}, 'P': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}}
0.5


/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
